# Parsing Journal of Cognitive Science article data Arthur already scraped (see GDrive)
Aim 1: Parse author & affiliation information for each article.

###### Tentative plans

LOOP THROUGH ALL ROWS: 
- Separate the author and affiliation data
- Create new row for each author in that paper, and denote whether that author was a corresponding author. 
- Also, keep the article name and index (row number) of the article in the original dataset.

In [2]:
import pandas as pd
import string as st

import matplotlib
matplotlib.use('Agg')
import numpy as np
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

In [3]:
data_file = '../data/171022_cogsci_journal_unprocessed.csv'
df_pd = pd.read_csv(data_file)
df = Table.read_table(data_file)

In [4]:
df

Unnamed: 0,article_name,year,volume,issue,pages,month,authors_and_details,num_of_authors,article_type,keywords,doi,references,num_of_references,references_authors,references_journals,references_nonjournal,references_titles_all,references_same_number,references_titles_journal,citedby,num_of_citedby,citedby_authors,citedby_journals,article_abstract
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Why Cognitive Science,1977,1,1,1-2,January,Allan Collins,1,Original Article,nan,10.1207/s15516709cog0101_1,nan,0,nan,nan,nan,nan,0,nan,"Marco Elio Tabacchi, Settimo Termini, “The human use of ...",12,Marco Elio Tabacchi/nSettimo Termini/n2017/nLoet Leydesd ...,Biophysical Chemistry/nJournal of the Association for In ...,nan
3,"An Overview of KRL, a Knowledge Representation Language",1977,1,1,3-46,January,"Daniel G. Bobrow,Corresponding author Xerox ...",2,Original Article,nan,10.1207/s15516709cog0101_2,"Anderson, J., & Bower, G. Human associative memory. W ...",55,"Anderson, J./nBower, G./n1973/nBobrow, D. G./nKaplan, R. ...",Artificial Intelligence/nCommunication of the Associatio ...,Human associative memory/nRepresentation and understandi ...,"Human associative memory/nGus, a frame driven dialog sys ...",1,"Gus, a frame driven dialog system/nA model and stack imp ...","Orlando Lopez-Cruz, Nini Johanna Garnica, Trends and App ...",278,Orlando Lopez-Cruz/nNini Johanna Garnica/n2018/n2015/nEr ...,Computational Linguistics/nExpert Systems with Applicati ...,"This paper describes KRL, a Knowledge Representation Lan ..."
4,Human and Computational Question Answering†,1977,1,1,47-73,January,Wendy LehnertCorresponding author Yale Unive ...,1,Original Article,nan,10.1207/s15516709cog0101_3,"Abelson, R. P. Concepts for representing mundane real ...",12,"Abelson, R. P./n1975/nCullingford, R. E./n1975/nMeehan, ...",American Journal of Computational Linguistics/nAmerican ...,Representation and understanding/nCenter for Human Infor ...,Concepts for representing mundane reality in plans/nAn a ...,0,An approach to the representation of mundane world knowl ...,"Colleen E. Crangle, Joyce Brothers Kart, A questions-bas ...",31,Colleen E. Crangle/nJoyce Brothers Kart/n2015/n2014/nSam ...,PeerJ/nTheory and Practice in Language Studies/nIBM Jour ...,The ability to answer questions about a text is the stro ...
5,Definite Descriptions and Semantic Memory†,1977,1,1,74-83,January,"Andrew Ortony,Corresponding author Universit ...",2,Original Article,nan,10.1207/s15516709cog0101_4,"Anderson, J. R., & Bower, G. H. Human associative mem ...",13,"Anderson, J. R./nBower, G. H./n1973/nAnderson, J. R./nHa ...",Cognitive Psychology/nAmerican Educational Research Jour ...,Human associative memory/nTranslations from the writings ...,Human associative memory/nIndividuation and reference in ...,1,Individuation and reference in memory: Proper names and ...,"Kristine H. Onishi, Gregory L. Murphy, Discourse model r ...",7,Kristine H. Onishi/nGregory L. Murphy/n2002/nBoaz Keysar ...,Language and Cognitive Processes/nJournal of Memory and ...,Subjects were exposed to sentences containing “direct” a ...
6,"Artificial Intelligence, Language, and the Study of Know ...",1977,1,1,84-123,January,"Professor Ira Goldstein,Corresponding author ...",2,Original Article,nan,10.1207/s15516709cog0101_5,"Anderson, J. R., & Bower, G. Human associative memory ...",48,"Anderson, J. R./nBower, G./n1973/n1975/nBrown, J. S./nMa ...",Linguistic Analysis/nArtificial Intelligence/nJournal of ...,Human associative memory/nRepresentation and understandi ...,Human associative memory/nProposal for knowledge-based i ...,1,Questions of form and interpretation/nSummary of MYCROFT ...,"Antoine C. M. van den Bogaart, Hans G. K. Hummel, Paul A ...",37,Antoine C. M. van den Bogaart/nHans G. K. Hummel/nPaul A ...,Professional Development in Education/nArtificial Intell ...,This paper studies the relationship of Artificial Intell ...
7,Induction of Augmented Trans

The 'author_and_details' column in the dataframe contains author and affiliation data.

In [5]:
df_ad = df['authors_and_details'][0:17]
print(df_ad)
print(type(df_ad[4]))

['nan' 'Allan Collins'
 'Daniel G. Bobrow,Corresponding author             Xerox Palo Alto Research Center/nTerry Winograd             Stanford University'
 'Wendy LehnertCorresponding author             Yale University'
 'Andrew Ortony,Corresponding author             University of Illinois at Urbana-Champaign/nRichard C. Anderson             University of Illinois at Urbana-Champaign'
 'Professor Ira Goldstein,Corresponding author             Massachusetts Institute of Technology/nSeymour Papert             Massachusetts Institute of Technology'
 'John R. AndersonCorresponding author             Yale University'
 'Jerome A. Feldman,Corresponding author             University of Rochester/nRobert F. Sproull             Xerox Palo Alto Research Center'
 'R. Bhaskar,             Carnegie-Mellon University/nHerbert A. SimonCorresponding author             Carnegie-Mellon University'
 'M. J. SteedmanCorresponding author             University of Sussex'
 'Yorick Wilks             Universi

In [6]:
df_ad[2].split('\n')#df['authors_and_details'][2]
print(range(len(df_ad)))

range(0, 17)


In [7]:
#create list to store all the orig rows
df_namesplit = []

for i in range(1, len(df_ad)):
        
    temp = df_ad[i].split('/n')
    df_namesplit.append(temp)
df_namesplit

[['Allan Collins'],
 ['Daniel G. Bobrow,Corresponding author             Xerox Palo Alto Research Center',
  'Terry Winograd             Stanford University'],
 ['Wendy LehnertCorresponding author             Yale University'],
 ['Andrew Ortony,Corresponding author             University of Illinois at Urbana-Champaign',
  'Richard C. Anderson             University of Illinois at Urbana-Champaign'],
 ['Professor Ira Goldstein,Corresponding author             Massachusetts Institute of Technology',
  'Seymour Papert             Massachusetts Institute of Technology'],
 ['John R. AndersonCorresponding author             Yale University'],
 ['Jerome A. Feldman,Corresponding author             University of Rochester',
  'Robert F. Sproull             Xerox Palo Alto Research Center'],
 ['R. Bhaskar,             Carnegie-Mellon University',
  'Herbert A. SimonCorresponding author             Carnegie-Mellon University'],
 ['M. J. SteedmanCorresponding author             University of Suss

In [89]:
def clearCorresponding(author):
    result = []
    if 'Corresponding author' in author:
        endIndex = author.find('Corresponding author')
        
        result.append(author[:endIndex])
        return result
    elif ',' in author:

        endIndex = author.find(',')
        result.append(author[:endIndex])
        return result
    else:
        return author
        
        """
        if 'Corresponding author' in num[0]:
            endind = num[0].find('Corresponding author')
            print(endind)
            print(num[0][:endind], '- ', num[1])
            Corr.append([num[0][:endind], '- ', num[1]]) """

In [90]:
results = Table(make_array("Corresponding Author", "Author", "Department/University"))
for i in range(len(df_namesplit)):
    num = df_namesplit[i]
    for j in range(len(num)):
        corr_auth = 0
        element = df_namesplit[i][j].split('             ')
        if len(element) > 1:
            author = element[0]
            #print (author[0])
            
            author = clearCorresponding(author)
            
            dept = element[1]
            
            if("Corresponding author" in df_namesplit[i][j]):
                corr_auth = 1
            #print(type(corr_auth))
            #newRow = make_array(corr_auth,author, dept)
            results = results.with_row([corr_auth,author, dept])
        else:            
            author = clearCorresponding(element[0])
            dept = "NaN"
            
            #newRow = make_array(corr_auth,author, dept)
            results = results.with_row([corr_auth,author, dept])
        
results.show()

Corresponding Author,Author,Department/University
0,Allan Collins,NaN
1,"Daniel G. Bobrow,",Xerox Palo Alto Research Center
0,Terry Winograd,Stanford University
1,Wendy Lehnert,Yale University
1,"Andrew Ortony,",University of Illinois at Urbana-Champaign
0,Richard C. Anderson,University of Illinois at Urbana-Champaign
1,"Professor Ira Goldstein,",Massachusetts Institute of Technology
0,Seymour Papert,Massachusetts Institute of Technology
1,John R. Anderson,Yale University
1,"Jerome A. Feldman,",University of Rochester


In [70]:
# index the row (e.g. 2nd) and split by /n character
df['authors_and_details'][2].split('/n')

['Daniel G. Bobrow,Corresponding author             Xerox Palo Alto Research Center',
 'Terry Winograd             Stanford University']

In [ ]:
# the 0th element contains the information of the first author
df['authors_and_details'][2].split('/n')[0]

In [ ]:
# further split author info to separate name and affiliation
au1 = df['authors_and_details'][2].split('/n')[0].split('             ')

In [ ]:
au1

In [ ]:
# find the corresponding author and delete that part of the string ',Corresponding author'
if 'Corresponding author' in au1[0]:
    endind = au1[0].find(',Corresponding author')
    print(endind)
    print(au1[0][:endind], '- ', au1[1])

In [ ]:
# make a new dataset


### The latter ones have a little more information
Let's go to index 1500

In [ ]:
df['authors_and_details'][1501].split('/n')

In [ ]:
df['article_name'][1501]